# Types of Machine Learning (Pre-Intro Module)

### Lesson Overview
**Audience:** High school students (no prior ML required)  
**Duration:** ~60 minutes  
**Goals:**
- Distinguish **Supervised**, **Unsupervised**, **Deep Learning**, and **Reinforcement Learning**.
- Explain when labeled vs. unlabeled data is used.
- Recognize real-world **applications** (robotics, images, language, LLMs).
- Appreciate why simpler models can sometimes beat complex ones.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
np.random.seed(42)

## Agenda
1. What is **Machine Learning**? (2 min)
2. **Supervised Learning** (labels) + Demo (12 min)
3. **Unsupervised Learning** (no labels) + Demo (10 min)
4. **Deep Learning** (neural networks) + Mini-demo (10 min)
5. **Reinforcement Learning** (learn from mistakes) + Mini-sim (12 min)
6. **Applications** & Choosing Model Complexity (9 min)
7. Exit Ticket (5 min)

## What is Machine Learning?
ML is about finding patterns in data to make predictions or decisions **without** being explicitly programmed for each situation.

## Supervised Learning
- You **must have a labeled dataset**: each example comes with the correct answer (the *label*).
- The model learns a mapping from inputs → label.
- Tasks: **classification** (cat vs. dog), **regression** (predict house price).

**Note (reference):** There is also **semi-supervised learning**, where you have **a few labeled** samples and **many unlabeled** ones. You use both to improve learning.

**Examples:**
- Email spam filter (label: spam / not-spam)
- Predicting exam score from study hours

### Q: Why are labels so important in supervised learning? Give one example of a label.

### A:
YOUR ANSWER HERE

In [ ]:
# Demo: Supervised classification on synthetic data
X, y = datasets.make_classification(
    n_samples=600, n_features=2, n_informative=2, n_redundant=0, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Try a simple model (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print("KNN accuracy:", round(knn.score(X_test, y_test), 3))

# Try a linear model (Logistic Regression)
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
print("Logistic Regression accuracy:", round(lr.score(X_test, y_test), 3))

# Visualize points
plt.figure()
plt.scatter(X_test[:,0], X_test[:,1], c=y_test)
plt.title("Supervised: test points (colored by true label)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

## Unsupervised Learning
- **No labels**: the goal is to **find structure** in unlabeled data.
- Tasks: **clustering** (group similar things), **dimensionality reduction** (compress features).

**Examples:**
- Grouping songs by listening patterns
- Segmenting customers by shopping behavior

### Q: How would you explain the difference between *classification* and *clustering* to a friend?

### A:
YOUR ANSWER HERE

In [ ]:
# Demo: Unsupervised clustering (KMeans) on unlabeled blobs
X_blobs, _ = datasets.make_blobs(n_samples=400, centers=3, n_features=2, cluster_std=1.0, random_state=42)

kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
labels = kmeans.fit_predict(X_blobs)
centers = kmeans.cluster_centers_

plt.figure()
plt.scatter(X_blobs[:,0], X_blobs[:,1], c=labels)
plt.scatter(centers[:,0], centers[:,1], marker='x', s=120)
plt.title("Unsupervised: KMeans clusters (colors) and centers (x)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

## Deep Learning
- Uses **neural networks** with multiple layers (can be applied to supervised or unsupervised tasks).
- Powerful for images, speech, and language.
- **Caution:** Some neural networks can be prone to **overfitting**. Sometimes the **simplest/easiest-to-explain** model is best!

**Applications:**
- Image classification (what’s in a photo?)
- Natural language processing (e.g., **Siri**) 
- Recent rise of **LLMs** (like ChatGPT) built on **Transformer** neural networks

### Q: Why might a simple model outperform a deep network on a small dataset?

### A:
YOUR ANSWER HERE

In [ ]:
# Mini-demo: A tiny neural net (MLP) on a simple dataset
X_moons, y_moons = datasets.make_moons(n_samples=600, noise=0.2, random_state=42)
X_tr, X_te, y_tr, y_te = train_test_split(X_moons, y_moons, test_size=0.25, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(16, 8), activation='relu', max_iter=1000, random_state=42)
mlp.fit(X_tr, y_tr)
print("MLP accuracy:", round(mlp.score(X_te, y_te), 3))

plt.figure()
plt.scatter(X_te[:,0], X_te[:,1], c=y_te)
plt.title("Deep Learning demo: test points (true labels)")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

## Reinforcement Learning (RL)
- Learn by **trial and error**: take an action → get **reward** → learn what to do next time.
- Goal: pick actions to **maximize long-term reward**.
- Applications: **Robotics**, game-playing, recommendation systems.

*(Reference idea: see an introductory article on Reinforcement Learning, e.g., a tutorial from a programming site.)*

### Q: How is RL different from supervised learning?

### A:
YOUR ANSWER HERE

In [ ]:
# Mini-simulation: epsilon-greedy multi-armed bandit (simple RL idea)
def bandit_sim(true_rewards, eps=0.1, steps=500):
    n = len(true_rewards)
    est = np.zeros(n)
    counts = np.zeros(n)
    rewards = []
    for t in range(steps):
        if np.random.rand() < eps:
            a = np.random.randint(n)  # explore
        else:
            a = np.argmax(est)        # exploit
        r = np.random.normal(true_rewards[a], 1.0)
        counts[a] += 1
        est[a] += (r - est[a]) / counts[a]
        rewards.append(r)
    return np.array(rewards)

true_rewards = [0.2, 0.5, 1.0]  # arm 2 is best on average
r = bandit_sim(true_rewards, eps=0.1, steps=500)
cum_avg = np.cumsum(r) / (np.arange(len(r)) + 1)
plt.figure()
plt.plot(cum_avg)
plt.title("RL demo: Cumulative average reward (epsilon-greedy)")
plt.xlabel("Step")
plt.ylabel("Avg reward")
plt.show()

## Applications Roundup
- **Robotics**: perception (vision), planning, control (often RL).
- **Image classification**: detect objects and scenes (often deep learning).
- **Natural language processing**: assistants like **Siri**, translation, chat.
- **LLMs**: recent rise of large language models (Transformers).
### Q: Pick one application above. Is it supervised, unsupervised, deep, or RL? Explain your choice.

### A:
YOUR ANSWER HERE

## Choosing Model Complexity
- Start simple; add complexity only if needed.
- More complex ≠ always better (risk of **overfitting**).
- Consider data size, noise, and explainability.

## Exit Ticket
1. Define **supervised** vs. **unsupervised** learning in your own words.
2. Why can deep learning overfit small datasets?
3. How does **reinforcement learning** use rewards to improve behavior?
4. Give one real-world application and classify it (supervised/unsupervised/deep/RL).

### A:
YOUR ANSWER HERE